In [ ]:
import torch
import torch
import torch.nn as nn

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn.functional as F

from torch.nn.utils.rnn import pad_sequence


/raid/home/rajivratn/anaconda3/envs/grpo/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model_name="meta-llama/Llama-3.2-1B"

# Initialize the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model.eval()
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


In [ ]:
sentences = ["QQUQ ZQIQ'Q QZ OZFQOXOTOX'OQX'Q Q QQ QHQUQUX QJKJQXA'ZQQOXQHOQHZQ'XC'QXQHY QQ Q UQ XQ QYQ QX FQOQCOQOZQUL YQZQZGOQH OXVFQ", "IT MAY BE NOTHING MORE THAN HANGING THE SCREEN DOOR CHOPPING THE WOOD OR DUSTING THE FURNITURE BUT IT WILL FURNISH HIM WITH SOME KIND OF ACTIVITY BECAUSE HE ENJOYS ACTION FOR ITS OWN SAKE AND BECAUSE WORK IS ONLY APPLIED ACTION THIS TYPE MAKES THE BEST WORKER"]
# Get the rewards using a LLM as Judge.
with torch.no_grad():
    inputs = tokenizer(sentences, return_tensors="pt", padding=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"] # 0 for padding
            
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    log_probs = F.log_softmax(outputs.logits, dim=-1)  # (B, T, V)
    
    target_ids = input_ids[:, 1:]
    log_probs = log_probs[:, :-1, :]
    token_log_probs = log_probs.gather(dim=2, index=target_ids.unsqueeze(-1)).squeeze(-1)
    target_mask = attention_mask[:, 1:]
    token_log_probs = token_log_probs * target_mask
    
rewards = token_log_probs.sum(dim=1) # per-sentence sum of log-probs shape: (B,)


In [ ]:
rewards

tensor([-359.8435, -228.8658])

: 